[语义搜索引擎指南](https://python.langchain.com/docs/tutorials/retrievers/)
介绍以下概念：
1. Documents and document loaders（文档和文档加载器）;
2. Text splitters 文本拆分器;
3. Embeddings 嵌入;
4. Vector stores and retrievers 向量存储和检索器。

In [23]:
# 用langSmith来记录链条内部的情况
import getpass
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = os.getenv('SMITH_API')

### Document包括三个数据：
- page_content：表示内容的字符串;
- metadata：包含任意元数据的 dict;
- id：（可选）文档的字符串标识符。

In [24]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [25]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "example_data\附件4.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))
print(docs[0].metadata)
docs[0].page_content[:100]

6
{'producer': 'Adobe PDF Library 19.21.90', 'creator': 'Acrobat PDFMaker 19 Word 版', 'creationdate': '2024-11-15T16:13:31+08:00', 'author': 'LENOVO', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2024-11-15T16:13:34+08:00', 'sourcemodified': 'D:20241115081232', 'subject': '', 'title': '', 'source': 'example_data\\附件4.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}


'地 83 级校友新生奖学金评选办法 \n地83 级校友专新生奖学金系我校地质工程专业83 级校友为庆贺我校120 周\n年校庆，吸引优秀学生填报我校地质学科学习，助力地质学科发展而设立。为便\n于该项奖金的'

### Split Text

In [26]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
# 拆分为1000个字符的块  包含200个字符重叠
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
# 这是一个Document
all_splits = text_splitter.split_documents(docs)

len(all_splits),all_splits[0]

(6,
 Document(metadata={'producer': 'Adobe PDF Library 19.21.90', 'creator': 'Acrobat PDFMaker 19 Word 版', 'creationdate': '2024-11-15T16:13:31+08:00', 'author': 'LENOVO', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2024-11-15T16:13:34+08:00', 'sourcemodified': 'D:20241115081232', 'subject': '', 'title': '', 'source': 'example_data\\附件4.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1', 'start_index': 0}, page_content='地 83 级校友新生奖学金评选办法 \n地83 级校友专新生奖学金系我校地质工程专业83 级校友为庆贺我校120 周\n年校庆，吸引优秀学生填报我校地质学科学习，助力地质学科发展而设立。为便\n于该项奖金的管理，特制定本评选办法。 \n一、奖励对象 \n西南交通大学地学学院全日制在读地质专业本科生及硕士研究生新生。 \n二、奖励人数及金额 \n每学年共设 5 个名额（研究生 2 名，本科生 3 名 ），3000 元/人 \n三、评选条件 \n（一）基本条件 \n1.本科新生奖：第一志愿填报西南交通大学地球科学与工程学院地质类专业，\n且高考分数超本省一本分数线前 3 位。 \n2.硕士研究生新生奖：第一志愿填报西南交通大学地质类专业的西南交通大\n学地质工程专业本科生，保研综合成绩第 1 名；考研录取综合成绩第 1 名。 \n四、评定程序 \n1.学生本人提出申请，地 83 级校友奖学金组委会评定； \n2.评审程序按《西南交通大学专项奖学金评选程序》进行。'))

### Embedding 看py脚本

In [27]:
from langchain_openai import OpenAIEmbeddings
import openai
api_key = os.getenv("OPENAI_API_KEY ")
base_url = os.getenv("OPENAI_API_BASE")
    
# 使用 openai v1 SDK 创建 client（必须使用 openai>=1.0.0）
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key=api_key,
    base_url=base_url
)
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"生成向量长度：{len(vector_1)}\n")
print(vector_1[:10])

生成向量长度：3072

[0.0010416125878691673, -0.013004467822611332, -0.006021607201546431, 0.01279629860073328, 0.008841078728437424, -0.025078294798731804, -0.02059653028845787, 0.03151930123567581, 0.008155344054102898, 0.004105223808437586]


### 矢量存储
LangChain VectorStore 对象包含用于将文本和对象添加到存储区以及**使用各种相似性指标查询**它们的方法。

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)  #实例化
ids = vector_store.add_documents(documents=all_splits)
ids

['84f80b17-ca16-419d-ab0e-43111f2af85e',
 '4794f19b-b7c0-4bf1-8180-d47b16fe4e9a',
 'a6a4ca3d-a93e-4ce8-af1a-e7756aaeb2cc',
 'fe4e30c0-5c73-4c8d-afca-acc728929eb2',
 'c0d917a1-5fac-41fd-8824-3df0830e9dc6',
 'c4362a29-8373-4e97-81b1-4636cd28fe41']

In [ ]:
# 与字符串查询的相似性返回文档
results = vector_store.similarity_search(
    "一等奖有多少人?"
)
results[0].page_content

'选。 \n4、学校专项奖学金评审委员会对获奖者进行复审并公示，向获奖学生颁发\n证书。'

In [39]:
# 返回相似性查询分数
results = vector_store.similarity_search_with_score("铁发奖学金有多少名额?")
doc, score = results[0]
doc, score

(Document(id='4794f19b-b7c0-4bf1-8180-d47b16fe4e9a', metadata={'producer': 'Adobe PDF Library 19.21.90', 'creator': 'Acrobat PDFMaker 19 Word 版', 'creationdate': '2024-11-15T16:13:31+08:00', 'author': 'LENOVO', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2024-11-15T16:13:34+08:00', 'sourcemodified': 'D:20241115081232', 'subject': '', 'title': '', 'source': 'example_data\\附件4.pdf', 'total_pages': 6, 'page': 1, 'page_label': '2', 'start_index': 0}, page_content='“铁发奖学金”评定办法 \n“铁发奖学金”系四川西南交大铁路发展股份有限公司为鼓励测绘学科学\n生热爱专业、勤奋学习、刻苦钻研、积极参与科研实验、努力成才、为祖国测绘\n事业贡献力量，而设的专项奖学金，从二〇二〇年九月开始实行。 \n一、评奖范围 \n西南交通大学地球科学与工程学院测绘工程、遥感、地信专业在校本科生及\n全日制硕士研究生。 \n二、奖励人数及金额 \n西南交通大学“铁发奖学金”每学年共设6 个名额，发放奖学金总额28000\n元。 \n本科生共 2 名（大三、大四各 1 名 ），3000 元/人。 \n硕士研究生共 2 名（研二、研三各 1 名 ），5000 元/人。 \n博士研究生共 2 名，6000 元/人。 \n三、评奖条件 \n1、热爱祖国、道德品质优良、遵守校规校纪和社会公德，无不良道德及违\n规记录。 \n2、学习勤奋刻苦、成绩优秀，参评学年无重修或补考； \n3、热爱所学专业，参评学年成绩位列本专业前 40%； \n4、参加科技活动及竞赛、在各类刊物发表文章及参与科研活动，具有良好\n的科研能力并取得一定的研究成果者优先考虑。 \n5、积极参加社团或社会志愿活动，具有活动组织经历或在活动组织中起

In [41]:
# 与嵌入式查询的相似性返回文档
embedding = embeddings.embed_query("一等奖有多少人?")
results = vector_store.similarity_search_by_vector(embedding)
results[0].page_content

'选。 \n4、学校专项奖学金评审委员会对获奖者进行复审并公示，向获奖学生颁发\n证书。'

## Retrievers

In [ ]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain

# 装饰器构建函数式链条
@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)

# 批量查询
# 使用 .batch([...]) 实现多个查询的批量处理。
retriever.batch(
    [
        "一等奖有多少人?",
        "铁发奖学金有多少名额?",
    ],
)

[[Document(id='a6a4ca3d-a93e-4ce8-af1a-e7756aaeb2cc', metadata={'producer': 'Adobe PDF Library 19.21.90', 'creator': 'Acrobat PDFMaker 19 Word 版', 'creationdate': '2024-11-15T16:13:31+08:00', 'author': 'LENOVO', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2024-11-15T16:13:34+08:00', 'sourcemodified': 'D:20241115081232', 'subject': '', 'title': '', 'source': 'example_data\\附件4.pdf', 'total_pages': 6, 'page': 2, 'page_label': '3', 'start_index': 0}, page_content='选。 \n4、学校专项奖学金评审委员会对获奖者进行复审并公示，向获奖学生颁发\n证书。')],
 [Document(id='4794f19b-b7c0-4bf1-8180-d47b16fe4e9a', metadata={'producer': 'Adobe PDF Library 19.21.90', 'creator': 'Acrobat PDFMaker 19 Word 版', 'creationdate': '2024-11-15T16:13:31+08:00', 'author': 'LENOVO', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2024-11-15T16:13:34+08:00', 'sourcemodified': 'D:20241115081232', 'subject': '', 'title': '', 'source': 'example_data\\附件4.pdf', 'total_pages': 6, 'page': 1, 'page_label': '2', 'start_index': 0}, page

In [ ]:
# 面向对象风格，不建议
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)
retriever.batch(
    [
        "一等奖有多少人?",
        "铁发奖学金有多少名额?",
    ],
)

[[Document(id='a6a4ca3d-a93e-4ce8-af1a-e7756aaeb2cc', metadata={'producer': 'Adobe PDF Library 19.21.90', 'creator': 'Acrobat PDFMaker 19 Word 版', 'creationdate': '2024-11-15T16:13:31+08:00', 'author': 'LENOVO', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2024-11-15T16:13:34+08:00', 'sourcemodified': 'D:20241115081232', 'subject': '', 'title': '', 'source': 'example_data\\附件4.pdf', 'total_pages': 6, 'page': 2, 'page_label': '3', 'start_index': 0}, page_content='选。 \n4、学校专项奖学金评审委员会对获奖者进行复审并公示，向获奖学生颁发\n证书。')],
 [Document(id='4794f19b-b7c0-4bf1-8180-d47b16fe4e9a', metadata={'producer': 'Adobe PDF Library 19.21.90', 'creator': 'Acrobat PDFMaker 19 Word 版', 'creationdate': '2024-11-15T16:13:31+08:00', 'author': 'LENOVO', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2024-11-15T16:13:34+08:00', 'sourcemodified': 'D:20241115081232', 'subject': '', 'title': '', 'source': 'example_data\\附件4.pdf', 'total_pages': 6, 'page': 1, 'page_label': '2', 'start_index': 0}, page